In [1]:
import numpy as np 
import matplotlib.pyplot as plt

# Our dilemma

In this exercise, we will look at flux-profile inversion. Our goal is to infer the friction velocity $u_{*} = \sqrt{- \overline{u'w'}}$ and roughness length $z_0$ using drone measurements of the mean horizontal wind speed $U(z)$ profile. 

For this, we will use the 'law of the wall':
$$ U(u_{*}, z_0, z) = \frac{u_{*}}{\kappa} \log(z/z_0) $$
which describes how the mean horizontal wind speed $U$ increases logarithmically with height $z$ in the neutral atmospheric surface layer. We want to infer the friction velocity $u_{*} = \sqrt{- \overline{u'w'}}$ and roughness length $z_0$ given drone measurements of the mean horizontal wind speed $U$ at various heights $z$. $\kappa = 0.4$ is the von Kárman constant. 

We can express our inverse problem as follows:
$$ y = \mathcal{F}(\mathbf{x}, z) + \epsilon $$
where $\epsilon$ is the noise term representing measurement error. 

What are $y$, $\mathbf{x}$, and $\mathcal{F}$? 

We will use the particle filter to infer the unknown parameters in a synthetic experiment.

# Generate synthetic data

In [2]:
def forward_mapping(u_s, z_0, z):
    """
    'law of the wall' describing idealized mean horizontal wind profiles
    in the atmospheric surface layer under neutral stability conditions.

    u_s:    friction velocity [m/s]
    z_0:    roughness length [m]
    z:      alititude [m]
    kappa:  von Karman constant [-]
    """
    kappa = 0.4 
    U = (u_s/kappa) * np.log(z/z_0) 
    return U

Now, we will generate our "thruth" for this synthetic experiment. 

In [ ]:
# our "thruth"
u_s_true = 0.5
z_0_true = 0.1

# vizualize the true mean horizontal wind speed profile
N_z = 5
z = np.logspace(0, 2, N_z) # n_z level evenly spaced on a log scale from 1 to 100 m
U_true = forward_mapping(u_s_true, z_0_true, z)

fig, ax = plt.subplots()
ax.plot(U_true, z, 'k')
ax.set_title('True wind speed profile')
ax.set_xlabel('Horizontal wind $U$ [m/s]')
ax.set_ylabel('Altitude $z$ [m]')
ax.set_ylim(0, 110)
ax.set_xlim(0, 10)

We need to create some synthetic data to mimic real, noisy, observations. We add some observation error to the true wind speed profile. We model the noise as a normal distribution with mean $\mu = 0$ and variance $\sigma^2 = \sigma^2_y$. 

We have:
$$ y_i = \mathcal{F}(\mathbf{x}, z_i) + \epsilon_i $$
where the additive Gaussian noise has the form $\epsilon_i \sim \mathcal{N}(0, \sigma_y^2)$.

In [ ]:
rng = np.random.default_rng() # create random number generator object

# fill in the standard deviation of the Gaussian noise
U_sigma = ...
U_obs = rng.normal(loc=U_true, scale=U_sigma, size=N_z)
U_obs = np.maximum(U_obs, 0) # Truncate to avoid negative wind speed observations.

fig, ax = plt.subplots()
ax.plot(U_true, z, 'k', label='Truth')
ax.scatter(U_obs, z, 50, color=(0.8,0.4,0), alpha=0.8, label='Obs')
ax.set_title('True wind speed profile and observations')
ax.set_xlabel('Horizontal wind $U$ [m/s]')
ax.set_ylabel('Altitude $z$ [m]')
ax.set_ylim(0, 110)
ax.set_xlim(0, 10)
ax.legend()

# Particle filter/smoother

We assume that we do not know the true windspeed profile, and we use our noisy observations and a particle filter to infer the friction velocity and roughness length. 

We start with defining a prior. Remember that all the particles are considered to have the same weight. 

In [ ]:
# fill in the number of particles
# generally, more particles gives more accuracy, but also larger computational time. 
N_particles = int(...)  

# fill in the bounds of the uniform prior
u_s_min, u_s_max = ..., ...
z_0_min, z_0_max = ..., ...

u_s_prior = rng.uniform(low=u_s_min, high=u_s_max, size=N_particles)
z_0_prior = rng.uniform(low=z_0_min, high=z_0_max, size=N_particles)

fig, ax = plt.subplots(1, 2)
ax[0].hist(u_s_prior, density=True)  # with density=True, the function returns a probability density, the area under the histogram integrates to 1.
ax[0].set_ylabel('Probability density')
ax[0].set_xlabel('$u_s$ [m/s]')
ax[0].set_xlim((u_s_min, u_s_max))
ax[1].hist(z_0_prior, density=True)
ax[1].set_ylabel('Probability density')
ax[1].set_xlabel('$z_0$ [m]')
ax[1].set_xlim((z_0_min, z_0_max))
fig.suptitle('Priors')
plt.tight_layout()


We can use the prior and our forward mapping to get prior predictions of the mean wind speed profile. 

$$\hat{y} = \mathcal{F}(\mathbf{x}, z) $$

where $\hat{y}$ are the prior predictions.

In [ ]:
def get_prior_prediction(u_s_prior, z_0_prior, z):
    u_s_prior = np.broadcast_to(u_s_prior, (N_z, N_particles)) 
    z_0_prior = np.broadcast_to(z_0_prior, (N_z, N_particles))
    z = np.broadcast_to(z[:, np.newaxis], (N_z, N_particles))
    U_prior_prediction = forward_mapping(u_s_prior, z_0_prior, z)
    return U_prior_prediction

U_prior_prediction = get_prior_prediction(u_s_prior, z_0_prior, z)

fig, ax = plt.subplots()
ax.plot(U_prior_prediction, z, alpha=0.4)
ax.set_xlabel('Horizontal wind U [m/s]')
ax.set_ylabel('Altitude z [m]')
ax.set_title('Prior predictions')

The particle filter/smoother uses importance sampling. We apply the bootstrap filter, whereby the proposal distribution is our prior. The weights can then be obtained through a simple likelihood ratio. The weight of the $j^{th}$ particle is given by:

$$ w_j = \frac{ p({\mathbf{y}}|\mathbf{x}_j) }{ \sum_{k=1}^{N_e} p(\mathbf{y}|\mathbf{x}_k )} $$

where $N_e$ is the number of particles and $\mathbf{y}$ is the batch of $N$ observations.

Modeling the measurement noise as a Gaussian distribution, leads to the following Gaussian likelihood:

$$ p(y_i | \mathbf{x}, z_i ) = \frac{1}{\sqrt{2 \pi \sigma_y}} \exp \left( - \frac{(y_i - \hat{y}_i)^2}{2 \sigma_y^2} \right)  $$

The weights can be computed by:

$$ w_j = \frac{ \exp \left( - \sum_{i=1}^{N} \frac{(y_i - \mathcal{F}(\mathbf{x}_j, z_i))^2}{2 \sigma_y^2} \right) }{ \sum_{k=1}^{N_e} \exp \left( - \sum_{i=1}^{N} \frac{(y_i - \mathcal{F}(\mathbf{x}_k, z_i))^2}{2 \sigma_y^2} \right) } $$

To ensure numerical stability, we first calculate the logarithm of the weigths. 

In [ ]:
def importance_sampling(y, y_hat, y_sigma):
    y = np.broadcast_to(y[:, np.newaxis], (N_z, N_particles))
    error = y - y_hat
    log_likelihood = -0.5 * np.sum(error**2, axis=0) / (y_sigma**2) 
    denominator = np.log( np.sum( np.exp( log_likelihood) ) )  # in scipy, you can use the function logsumexp
    log_weights = log_likelihood - denominator
    weights = np.exp(log_weights)
    weights = weights / np.sum(weights)  # to ensure normalized weights
    return weights

weights = importance_sampling(U_obs, U_prior_prediction, U_sigma)

fig, ax = plt.subplots(1, 2)
ax[0].hist(u_s_prior, density=True, weights=weights)  # plot the same particles but now with updated weights
ax[0].set_ylabel('Probability density')
ax[0].set_xlabel('$u_s$ [m/s]')
ax[0].set_xlim((u_s_min, u_s_max))
ax[1].hist(z_0_prior, density=True, weights=weights)
ax[1].set_ylabel('Probability density')
ax[1].set_xlabel('$z_0$ [m]')
ax[1].set_xlim((z_0_min, z_0_max))
fig.suptitle('Weighted distributions')
plt.tight_layout()

In [ ]:
u_s_mean = np.sum(weights * u_s_prior)
z_0_mean = np.sum(weights * z_0_prior)

u_s_sigma = np.sqrt(np.sum(weights * (u_s_prior - u_s_mean)**2))
z_0_sigma = np.sqrt(np.sum(weights * (z_0_prior - z_0_mean)**2))

print(f"posterior mean u_s = {u_s_mean} m/s")
print(f"posterior sigma u_s = {u_s_sigma} m/s")

print(f"posterior mean z_0 = {z_0_mean} m")
print(f"posterior sigma z_0 = {z_0_sigma} m")

## Effective sample size 

The effective sample size is a metric for the accuracy of the estimates. In the ideal case we have that $w_j = 1/N_e$ for all particles $j$ so that ESS = $N_e$. In the worst case, the so-called degenerate case, all the weight is on one particle so that ESS = 1. 

$$ ESS = \frac{1}{\sum_j w_j^2} $$

In [ ]:
def get_ESS(weights):
    N_eff =1/np.sum(weights**2)
    return N_eff

N_eff = get_ESS(weights)

print(f"ESS = {N_eff}")

# Resampling

It is possible to resample the particles based on their weight. Hereby we copy high weight particles and remove the low weight particles. In this way, we end up with equally weighted resampled particles. 

We can then:
- Use simple arithmetic (instead of weighted) means,
- We can use the resampled particles as the prior for a (possible) next iteration of the particle filter with new data. If the ESS is low, it is common to add some 'jitter' to the particles, so-called rejuvenation. This is to prevent the collapse of the weights. You just have to make sure that the distribution remains within the bounds of the initial prior.

In [ ]:
# resampling
selection_index = rng.choice(N_particles, size=N_particles, p=weights)  # replace = True by default
u_s_resampled = u_s_prior[selection_index]
z_0_resampled = z_0_prior[selection_index]

# rejuvenation
u_s_new_prior = rng.normal(loc=u_s_resampled, scale=0.01)  # scale is the standard deviation of the Gaussian added noise
u_s_new_prior = np.maximum(u_s_min, np.minimum(u_s_new_prior, np.ones(N_particles)*u_s_max))  # make sure that the distribution stays within bounds
z_0_new_prior = rng.normal(loc=z_0_resampled, scale=0.01)  
z_0_new_prior = np.maximum(np.ones(N_particles)*z_0_min, np.minimum(z_0_new_prior, np.ones(N_particles)*z_0_max))

fig, ax = plt.subplots(1, 2)
ax[0].hist(u_s_resampled, density=True)  
ax[0].set_ylabel('Probability density')
ax[0].set_xlabel('$u_s$ [m/s]')
ax[0].set_xlim((u_s_min, u_s_max))
ax[1].hist(z_0_resampled, density=True)
ax[1].set_ylabel('Probability density')
ax[1].set_xlabel('$z_0$ [m]')
ax[1].set_xlim((z_0_min, z_0_max))
fig.suptitle('Resampled distributions')
plt.tight_layout()

fig, ax = plt.subplots(1, 2)
ax[0].hist(u_s_new_prior, density=True)  
ax[0].set_ylabel('Probability density')
ax[0].set_xlabel('$u_s$ [m/s]')
ax[0].set_xlim((u_s_min, u_s_max))
ax[1].hist(z_0_new_prior, density=True)
ax[1].set_ylabel('Probability density')
ax[1].set_xlabel('$z_0$ [m]')
ax[1].set_xlim((z_0_min, z_0_max))
fig.suptitle('Rejuvenated distributions')
plt.tight_layout()

## Exercise

Create another set of synthetic observations and run the particle filter with this new set. Can we further constrain the probability distribution? Do we approach the thruth? What is the ESS?

In [ ]:
# Define another set of wind speed profile observations and apply the particle filter. 
# You can use the posterior of the previous iteration as prior, or come up with a different prior.